# Noize net
The following notebook contains the code used for both an LSTM and RNN used for investigating the feasibility of music generation using an RNN. 

Note that berevity of code both the RNN and LSTM are implimented in one code base. This does mean that the code is somewhat more complicated and necessitates the use of "LSTMBool" which defines throughout the code if operations will take place for an LSTM or RNN. 

Note to reader: This notebook is a refactor of the main python file noizenet.py


# Imports

First we must import all the necessary dependencies.

Then check if we can run on a GPU.

Additionally we can define the scaler for scaling input data and inverting a scale on output data. 

In [7]:
import matplotlib   
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import librosa as lib
import os
import soundfile as sf #For writing
import matplotlib.pyplot as plt
import gc
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import sklearn as skl
import pandas as pd
import utils
import librosa.display
import noise


# torch.autograd.detect_anomaly(True) #Check for errors and return a stack trace. (Used to debug nan loss)

# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

#Print if we are able to use a GPU
if(train_on_gpu):
    print('Processing on GPU.')
else:
    print('No GPU available.')

scaler = sklearn.preprocessing.MinMaxScaler()

Processing on GPU.


# Helper functions
Bellow we define some functions that will be used later in this work

## Time to FT
Convert an array of time/amplitude values to frequency domain

In [8]:
def time_to_fft(time_domain):
    # Compute FFT
    fft = np.fft.fft(time_domain)
    # Concatinate real and imaginary values
    new_input = np.concatenate((np.real(fft_block), np.imag(fft_block)))
    return new_input

## FT to Time
Convert an array of fourier values to the time domain

In [9]:
def fft_to_time(ft_domain):
    # check if the input can be divinded in two and assign as needed
    if ft_domain.shape[0] % 2 == 0:
        num_elems = (int)(blocks_ft_domain.shape[0] / 2)
    else:
        ft_domain = ft_domain[0:-1]
        num_elems = (int)(ft_domain.shape[0] / 2)
    # Get real part 
    real = ft_domain[0:num_elems]
    # Get imaginary part
    imag = ft_domain[num_elems:]
    # Recompose real and Im parts
    composition = real + 1.0j * imag
    # ifft back to time domain
    time = np.fft.ifft(composition)
    return time

## Batching function 
This function is rarely used in this work but some testing was done with batches to check their performance.

In [10]:
def get_batches(arr, batch_size, seq_length):
    #Note this code was adapted fome Udacity course on machine learning. 
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough elements to fill a batch
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

# Defining the model
Then we can define the Model using the pytorch class definition.

The model contains the following layers:
* LSM or RNN layer 
* Dropout Layer
* Fully connected layer for translating oyutput

After defining the layers in the model we can define a forward function used for the formward pass during training and prediction. 
* Given c0 (Or None for RNN), the hidden state and some imput x.
* Pass the hidden state and input through the RNN layer and recieve some output
* Selectively choose values to drop (Note this only happens for hidden layers > 1)
* Pass the result through the fully connected layer to obtain the final result
* Return the prediction and hidden states

Additionally we define a helper funcation to generate the hidden states for an LSTM

In [11]:
class NoizeNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, LSTMBool, dropout_prob):
        super(NoizeNet, self).__init__()

        self.LSTMBool = LSTMBool  
        self.hidden_dim = hidden_dim
        self.num_layers = n_layers
        self.output_size = output_size
        self.input_size = input_size
        self.dropout_prob = dropout_prob


        #TODO: Test RNN vs LSTM
        if LSTMBool:
            self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=dropout_prob,batch_first=True)
            self.hidden = (torch.zeros(1,1,self.hidden_dim).cuda(), torch.zeros(1,1,self.hidden_dim).cuda()) #We need a tuple for a LSTM
        else:
            # define an RNN with specified parameters
            # batch_first means that the first dim of the input and output will be the batch_size
            self.rnn = nn.RNN(input_size, hidden_dim, n_layers, dropout=dropout_prob, batch_first=True)

        self.dropout = nn.Dropout(dropout_prob)

        # last, fully-connected layer
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden, c0=None):
        batch_size = x.size(0)
        if (train_on_gpu):
            x.cuda()
        else:
            x.cpu()
        

        if self.LSTMBool:
            h_0 = hidden
            c_0 = c0
            #Rename variables as convention
            state = (h_0, c_0)
            # Propagate input through LSTM
            r_out, (hn, cn) = self.lstm(x, state) #lstm with input, hidden, and internal state
            hidden = hn
        else:
            # get RNN outputs
            r_out, hidden = self.rnn(x , hidden)

        r_out = self.dropout(r_out) #Dropout

        # shape output to be (batch_size*seq_length, hidden_dim)
        if (train_on_gpu):
            hidden.cuda()
            
        r_out = r_out.view(-1, self.hidden_dim)

        # get final output
        output = self.fc(r_out)

        if self.LSTMBool:
            return output, (hidden, cn)
        else:
            return output, hidden
        
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        #THIS CODE IS FROM UDACITY
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM

        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = torch.autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)).cuda() #hidden state
            c0 = torch.autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)).cuda() #internal state
        else:
            hidden = torch.autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)).cuda() #hidden state
            c0 = torch.autograd.Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)).cuda() #internal state

        return hidden, c0

# Training
We can then define the training function. This function takes in the model, n_steps which is a bit of a misnoma as it is used as the number of full "frame" steps that will complete the impute sequence. An array of genreTracks which are translated into file names. Using AUDIO_DIR and file names we can read in the input songs. Step size is used to determine how large of a step to take in the training data. Duration is the duration of the input song to read in. Number of tracks is the number of songs that we will train on. Clip is the value of the griadient clipping. 

In this work we vary the training procedue among trails. However the procedure remains unchanged and is as follows:
* Ensure model is in training mode
* load validation data and perform and needed transformations/scaling
* Initialize hidden layers
* Loop over a number of input tracks
* At each loop iteration
    * Load and transform input training data
    * Define the needed variables
    * Check if input data contains a NaN value
    * For some number of steps that was calculated earlier
        * For some number of epochs 
            * Select some data
            * Ensure data has a shift in input and target
            * Convert to tensors and give the input a batch size dimension of one
            * Move data to GPU if we can
            * Get a prediction frome the LSTM or RNN
            * Detach hidden states from history
            * Zero gradients
            * Calculate loss and add it to the plot array
            * Perform backward step
            * Clip gradients
            * Take an optimizer step to optimize weights
            * Perform validation (I will not walk through this as it is the same procedure in eval mode)
    * Clean memory if the model is large
* Plot losses
* Return the trained model

In [12]:
def train(noizeNet, n_steps, AUDIO_DIR, genreTracks, LSTM ,step_size=1, duration=5, numberOfTracks=1, clip=5, fft_bool=False):
    fileCount = 0 #Used for displaying the file that is currently being trained on
    noizeNet.train() #Set the model to training mode
    lossArr = [] #Array used to plot loss over time
    val_losses = []
    if(train_on_gpu):
        noizeNet.cuda() #Move the model to GPU if available

    val_file = utils.get_audio_path(AUDIO_DIR, genreTracks[-1])
    val_data, sr = lib.load(val_file, mono=True, duration = duration)
    if fft_bool:
        print("We are training on FFT data")
        val_data = time_to_fft(val_data)
    val_data = scaler.fit_transform(val_data.reshape(-1,1)) #Scale data
    hidden = None
    c0 = None
    hidden, c0 = noizeNet.init_hidden(1)
    #Loop over all the files in our filtered list
    for id in genreTracks: 
        #Stop training after n files
        fileCount+=1
        if(fileCount > numberOfTracks):
            break 

        filename = utils.get_audio_path(AUDIO_DIR, id) #Get the actual path to the file from the id
        
        fileData, sr = lib.load(filename, mono=True, duration = duration)
        fileData = fileData.reshape(-1,1)
        if fft_bool:
            fileData = time_to_fft(fileData)
        fileData = scaler.fit_transform(fileData)

        data = fileData
        batch_size = (int)(duration*sr/n_steps) #Find the size of the window that slides across the input song
        number_of_steps = (int)(duration*sr)-batch_size #Assumes step size of one as a larger step size produced poor results

        if(np.isnan(np.sum(fileData))):
            print("NAN ON FILE:\t", filename)
            break
        
        for batch_i in (range(0,number_of_steps, step_size)):
            for e in range(0,1):
                # for batch_i in (range(0,number_of_steps, step_size)):
                # if LSTM:
                #     (hidden, c0) = noizeNet.init_hidden(batch_size)
                # for x, y in get_batches(fileData, 500, 50):
                
                # defining the training data
                data = fileData[(batch_i): batch_size + batch_i]
                # data = np.resize(data,((batch_size), 1)) 
                # data = data.reshape(batch_size,1)  
                # data = data.reshape(len(data),1)    
                x = data[:-1] #Select all but the last element in the input data
                y = data[1:] #Select all but the first element in the input data. Essentially a forward shift in time

                # convert data into Tensors
                x_tensor = torch.Tensor(x).unsqueeze(0)  # unsqueeze gives a 1, batch_size dimension
                y_tensor = torch.Tensor(y)
                if(train_on_gpu):
                        x_tensor, y_tensor = x_tensor.cuda(), y_tensor.cuda()
                
                if LSTM:
                    prediction, (hidden, c0) = noizeNet(x_tensor, hidden, c0) #LSTM!
                else:
                    # outputs from the rnn
                    prediction, hidden = noizeNet(x_tensor, hidden)
                
                ## Representing Memory ##
                # make a new variable for hidden and detach the hidden state from its history
                # this way, we don't backpropagate through the entire history
                hidden = hidden.data

                if LSTM:
                    c0 = c0.data #LSTM!


                # zero gradients
                optimizer.zero_grad()

                # calculate the loss
                
                loss = criterion(prediction, y_tensor)
                # if(np.isnan(loss)):
                #     break
                lossArr.append(loss.item())
                # perform backprop and update weights
                loss.backward()
                
                torch.nn.utils.clip_grad_value_(noizeNet.parameters(), clip) #Clip gradient
                optimizer.step()


                #Validation step
                if(int((batch_i)) % 1000 == 0):
                    if LSTM:
                        (val_h, val_c) = noizeNet.init_hidden(1)
                    else:
                        val_h = None

                    
                    noizeNet.eval()
                    val_x = val_data[:-1]
                    val_y = val_data[1:]

                    

                    for e in range(0,1):
                        val_x, val_y = torch.from_numpy(val_x), torch.from_numpy(val_y)
                        
                        
                        inputs, targets = val_x.reshape(-1,1).unsqueeze(0), val_y.reshape(-1,1)
                        if(train_on_gpu):
                            inputs, targets = inputs.cuda(), targets.cuda()
                        if LSTM:
                            output, (val_h, val_c) = noizeNet(inputs, val_h, val_c)
                        else:
                            output, val_h = noizeNet(inputs, val_h)
                        val_loss  = criterion(output, targets)

                        val_losses.append(val_loss.item())

                        if LSTM:
                            val_h = val_h.data
                            val_c = val_c.data
                        else:
                            val_h = val_h.data


                    print("Training Progress:\t", round((fileCount*e/(number_of_steps*numberOfTracks))*100, 2), "%"  , sep="")
                    print("P:\t", prediction.cpu().data.numpy().flatten()[-5:],"\nY:\t", y[-5:].flatten() ,"\nX:\t" , x[-5:].flatten() ,sep="")
                    print('Training Loss: ', loss.item(), "Validation loss: ", val_loss.item() , "\t num:", batch_i, "\t File:", fileCount)
                    noizeNet.train() # reset to train mode after validation
            
            #Clean unused variables to ensure memory is kept free
            # del prediction
            # del x_tensor
            # del y_tensor
            # del data
            # del x
            # del y
            # gc.collect()
        fig, ax = plt.subplots(nrows=2)
        ax[0].plot(lossArr)
        ax[0].set(title="Training loss", ylabel="Loss", xlabel="Epochs")
        ax[1].plot(val_losses)
        ax[1].set(title="Validation loss", ylabel="Loss", xlabel="Epochs")
        plt.show()
    return noizeNet

# Prediction and Music generation

Then we define the prediction function. 

In this work we have two prediction schemes that produce equivalent results. I will explain them independently. 

## Prediction scheme 1 
This prediction sheme functions as follows
* Recieve the model, prediction seed track, duration of seed, prediction duration and LSTM bool
* Read in the seed data and perform any needed scaling/transformation
* Select some range of the seed and obtain a prediction from the model
* Record the last element in the prediction
* Append this predicted value to a music array that will become the generated song
* Shift the data window to exclude an early data point and append the predicted value
* Repeat this procedure until we have predicted enough points for the desired music output. 

In [13]:
def predict(noizeNet, genreTrack ,duration=1, n_steps=30, LSTMBool=False, predictDuration = 30, step_size=1):
    print("PREDICTING...")
    noizeNet.eval()
    hidden = None
    
    if LSTMBool:
        c0 = None
        (hidden, c0) = noizeNet.init_hidden(1)
    filePath = utils.get_audio_path(AUDIO_DIR, genreTrack) #Get the actual path to the file from the id
    y, sr = lib.load(filePath, mono=True, duration = duration)
    # y = time_to_fft(y)
    y = scaler.fit_transform(y.reshape(-1,1))

    data = y
    

    # data = np.random.normal(-1,1,y.shape)
    # data = genPerlin(np.linspace(0,1,y.size))
    batch_size = (int)(sr*duration/n_steps)



    number_of_steps = (int)(sr*predictDuration/step_size)

    music = []
    next = data[batch_size-1]

    sf.write('/home/liam/Desktop/University/2021/MAM3040W/thesis/writeup/code/predictionSeed.wav', np.append( data[step_size: batch_size], next), sr,format="WAV")
    print("BATCH SIZE:", batch_size ,sep="\t")
    print("NUMBER OF STEPS:", number_of_steps , sep="\t")
    # data = data[0: batch_size-1]
    hidden = None
    c0 = None
    hidden, c0 = noizeNet.init_hidden(1)
    for batch_i in (range(0, number_of_steps)):
        if(train_on_gpu):
            noizeNet.cuda()
        data = data[batch_i: batch_size-1 + batch_i]
        data = np.append(data,next)
        x = data.reshape(-1,1)

        print(data.shape)
        if(np.isnan(np.sum(data))):
            print("data contains NAN", data)

        x_tensor = torch.Tensor(x).unsqueeze(0)  # unsqueeze gives a 1, batch_size dimension
        if(train_on_gpu):
                x_tensor = x_tensor.cuda()

        if(LSTMBool):
            prediction, (hidden, c0) = noizeNet(x_tensor, hidden, c0)
            c0 = c0.data
        else:
            prediction, hidden = noizeNet(x_tensor, hidden)
        hidden = hidden.data
        
        
        music = np.append(music,(prediction.cpu().data.numpy().flatten())[-1])
        next = prediction.cpu().data.numpy().flatten()[-1]
        
        if(int((batch_i)) % 100 == 0):
            # fig, ax = plt.subplots(nrows=2)
            # ax[0].plot(prediction.cpu().data.numpy())
            # ax[1].plot(scaler.inverse_transform(prediction.cpu().data.numpy()))
            # print(np.average(prediction.cpu().data.numpy()))
            # print(prediction.cpu().data.numpy().flatten()[-1:])
            # print(x)
            # ax[1].plot(fft_to_time(prediction.cpu().detach().numpy()))
            # plt.show()
            
            print("PROGRESS:\t", round(((batch_i)/number_of_steps)*100, 2), "%"  , sep="")
            print("Prediction dimensions:\t", prediction.cpu().size(), "\t" ,prediction.cpu().data.numpy().flatten().size, "\nMusic dimensions:\t", music.size ,sep="")
       
    # print(fft_to_time(music))  
    print(music)
    music = scaler.inverse_transform(music.reshape(-1,1))
    # music = prediction.cpu().detach().numpy()
    # music = abs(fft_to_time(music))#.astype('float32')
    # print(music)
    sf.write('/home/liam/Desktop/University/2021/MAM3040W/thesis/writeup/code/outputSoundFile.wav', (music), 22050,format="WAV")
    time_steps = np.linspace(0, len(music), len(music))
    plt.plot(time_steps, music,"b.",  markersize=0.1)
    plt.show()
    return music

## Prediction Scheme 2
This scheme is similar to the previous one, however, more efficient as we process the seed element by elements and then pass the hidden states to a new function that will begin predicting new values from the last predicted value from the seed and hidden states. 

In [14]:
def predict2(noizeNet, genreTrack ,duration=1, n_steps=30, LSTMBool=False, predictDuration = 30, step_size=1):
    print("PREDICTING...")
    noizeNet.eval()

    if LSTMBool:
        hidden, c0 = noizeNet.init_hidden(1)
    else:
        hidden = None
    filePath = utils.get_audio_path(AUDIO_DIR, genreTrack) #Get the actual path to the file from the id
    y, sr = lib.load(filePath, mono=True, duration = duration)
    sf.write('/home/liam/Desktop/University/2021/MAM3040W/thesis/writeup/code/Predict2Seed.wav', y, 22050,format="WAV")
    y = scaler.fit_transform(y.reshape(-1,1))
 
    batch_size = (int)(sr*duration/n_steps)

    number_of_steps = (int)(sr*duration/step_size)
    music = []
    for batch_i in (range(0, number_of_steps)):
        if(train_on_gpu):
            noizeNet.cuda()
        data = y[batch_i]
        data = data.reshape(-1,1)
        x = data


        x_tensor = torch.Tensor(x).unsqueeze(0)  # unsqueeze gives a 1, batch_size dimension
        if(train_on_gpu):
                x_tensor = x_tensor.cuda()

        if(LSTMBool):
            prediction, (hidden, c0) = noizeNet(x_tensor, hidden, c0)
            c0 = c0.data
        else:
            prediction, hidden = noizeNet(x_tensor, hidden)
        hidden = hidden.data
        
        print(prediction.cpu().data.numpy().shape)
        music = np.append(music,(prediction.cpu().data.numpy().flatten())[-step_size:])
        if(int((batch_i)) % 1000 == 0):
            print("PROGRESS:\t", round(((batch_i)/number_of_steps)*100, 2), "%"  , sep="")
            print("Prediction dimensions:\t", prediction.cpu().size(), "\t" ,prediction.cpu().data.numpy().flatten().size, "\nMusic dimensions:\t", music.size ,sep="")
       
    sf.write('/home/liam/Desktop/University/2021/MAM3040W/thesis/writeup/code/outputSoundFilePredict2.wav', (music), 22050,format="WAV")
    time_steps = np.linspace(0, len(music), len(music))
    plt.plot(time_steps, music,"b.",  markersize=1)
    plt.show()
    if LSTMBool:
        return music, (hidden, c0)
    else: 
        return music, hidden

In [15]:
def predict3(noizeNet, seeded ,duration=1, n_steps=30, LSTMBool=False, predictDuration = 30, step_size=1, hidden = None, c0 = None):
    print("PREDICTING...")
    noizeNet.eval()
    number_of_steps = (int)(22050*predictDuration/step_size)
    music = []

    for batch_i in (range(0, number_of_steps)):
        if(train_on_gpu):
            noizeNet.cuda()
        seeded = np.array([seeded]).reshape(-1,1)
        data = seeded
        if x - seeded[0] < 0.001:
            data = seeded*np.random.normal(-1,1)
        data = data.reshape(-1,1)
        x = data
        
        x_tensor = torch.Tensor(x).unsqueeze(0)  # unsqueeze gives a 1, batch_size dimension
        print("x_tensor",x_tensor)
        if(train_on_gpu):
                x_tensor = x_tensor.cuda()

        if (LSTMBool):
            prediction, (hidden, c0) = noizeNet(x_tensor, hidden, c0)
            c0 = c0.data
        else:
            prediction, hidden = noizeNet(x_tensor, hidden)
        hidden = hidden.data
        
        seeded = prediction.cpu().data.numpy()[-1:step_size:]
        music = np.append(music,(prediction.cpu().data.numpy().flatten())[-step_size:])
        if(int((batch_i)) % 10000 == 0):
            print("PROGRESS:\t", round(((batch_i)/number_of_steps)*100, 2), "%"  , sep="")
            print("Prediction dimensions:\t", prediction.cpu().size(), "\t" ,prediction.cpu().data.numpy().flatten().size, "\nMusic dimensions:\t", music.size ,sep="")
       
    # print(fft_to_time(music))  
    # print(music)
    music = scaler.inverse_transform(music)
    # music = music.cpu().data.numpy().flatten()
    # music = abs(fft_to_time(music))#.astype('float32')
    print(music)
    sf.write('/home/liam/Desktop/University/2021/MAM3040W/thesis/writeup/code/outputSoundFilePrediction3.wav', (music), 22050,format="WAV")
    time_steps = np.linspace(0, len(music), len(music))
    plt.plot(time_steps, music,"b.",  markersize=1)
    plt.show()
    return music

## Saving function
A helper function used for naming a model by it's hyer parameters.

In [16]:
def generateModelName(n_steps = 30, print_every = 5, step_size =  1, duration = 5, numberOfTracks = 1, clip = 5, LSTMBool=False, hidden_dim=50,n_layers=1):
    return "n_steps="+ str(n_steps) + "__" +"print_every="+ str(print_every) + "__" +"step_size="+ str(step_size) + "__" +"duration="+ str(duration) + "__" +  \
    "numberOfTracks="+ str(numberOfTracks) + "__" +  "clip="+ str(clip) + "__" +  "LSTMBool="+ str(LSTMBool) + "hidden_dim="+ str(hidden_dim) + "__" +"n_layers="+ str(n_layers) +"__"+ "lr=" + str(lr) + ".pt"


# Hyper parameters
Before we start training we must declair the hyperparameters and instantiate the model.

I believve all of these variables should be self explanatory. Note this is also where we choose if the miodel will be an LSTM or RNN.

In [17]:
# decide on hyperparameters
# n_steps = 1
input_size=1
output_size=1
hidden_dim=100
n_layers=2
LSTMBool = True
dropout_prob = 0.5

# instantiate an RNN
noizeNet = NoizeNet(input_size, output_size, hidden_dim, n_layers, LSTMBool, dropout_prob)
print(noizeNet)

# MSE loss and Adam optimizer with a learning rate of 0.01
criterion = nn.MSELoss()
lr=0.0001
# criterion = nn.L1Loss()
# optimizer = torch.optim.Adam(noizeNet.parameters(), lr=lr)
# optimizer = torch.optim.SGD(noizeNet.parameters(),lr=lr)
optimizer = torch.optim.Adadelta(noizeNet.parameters())

NoizeNet(
  (lstm): LSTM(1, 100, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=100, out_features=1, bias=True)
)


# Music filtering by genre

Here we just filter all the data in FMA for some chosen genre. 

In [18]:
#Get metadata for fma dataset
AUDIO_DIR = "data/fma_small/"

tracks = utils.load('data/fma_metadata/tracks.csv')

small = tracks['set', 'subset'] <= 'small'
genre1 = tracks['track', 'genre_top'] == 'Instrumental'
# genre2 = tracks['track', 'genre_top'] == 'Hip-Hop' #We can set multilpe genres bellow as (genre1 | genre2)
genreTracks = list(tracks.loc[small & (genre1),('track', 'genre_top')].index)

# Training variables

Initialise the variables needed for training the network

In [19]:
#Set if we want to train new model or load and predict with saved model
TRAIN = True

n_steps = 30 #The number of full frame steps to be taken to complete training
print_every = 5 
step_size =  1 #The step size taken by each training frame 
duration = 1 #The duration of the training segment
predictDuration = 1 #The duration of the predicted song is seconds
numberOfTracks = 1 #The number of tracks to be trained on
clip = 1 #Gradient clipping
seedDuration = 1

# Begin Training or load model and predict
Then we can call our training and prediction functions as needed. 

In [ ]:
if TRAIN:
    print("TRAINING...")
    trained_rnn = train(noizeNet, n_steps,AUDIO_DIR, genreTracks, LSTMBool ,step_size=step_size, duration = duration, numberOfTracks=numberOfTracks, clip=clip)
    torch.save(trained_rnn.state_dict(), "/home/liam/Desktop/University/2021/MAM3040W/thesis/writeup/" + generateModelName(n_steps, print_every, step_size, duration, numberOfTracks, clip, LSTMBool, hidden_dim,n_layers))
    duration = seedDuration
    # predict(trained_rnn, genreTracks[-1] ,duration=duration, n_steps=n_steps, LSTMBool=LSTMBool, predictDuration = predictDuration, step_size=step_size)
    if LSTMBool:
        predicted, (hidden, c0) = predict2(trained_rnn, genreTracks[-2] ,duration=duration, n_steps=n_steps, LSTMBool=LSTMBool, predictDuration = predictDuration)
        predict3(noizeNet, predicted[-1] ,duration=duration, n_steps=n_steps, LSTMBool=LSTMBool, predictDuration = predictDuration,  hidden=hidden, c0 =c0)
    else: 
        predicted, hidden = predict2(trained_rnn, genreTracks[-2] ,duration=duration, n_steps=n_steps, LSTMBool=LSTMBool, predictDuration = predictDuration)
        predict3(noizeNet, predicted[-1] ,duration=duration, n_steps=n_steps, LSTMBool=LSTMBool, predictDuration = predictDuration,  hidden=hidden)
    

    

else:
    # instantiate an RNN
    noizeNet = NoizeNet(input_size, output_size, hidden_dim, n_layers, LSTMBool, dropout_prob)
    noizeNet.load_state_dict(torch.load("/home/liam/Desktop/University/2021/MAM3040W/thesis/writeup/code/good model/n_steps=1__print_every=5__step_size=1__duration=10__numberOfTracks=100__clip=5__LSTMBool=Truehidden_dim=200__n_layers=1__lr=0.0001.pt"))
    duration = seedDuration
    if LSTMBool:
        predicted, (hidden, c0) = predict2(noizeNet, genreTracks[-2] ,duration=duration, n_steps=n_steps, LSTMBool=LSTMBool, predictDuration = predictDuration)
        predict3(noizeNet, predicted[-1] ,duration=duration, n_steps=n_steps, LSTMBool=LSTMBool, predictDuration = predictDuration,  hidden=hidden, c0 =c0)
    else: 
        predicted, hidden = predict2(noizeNet, genreTracks[-2] ,duration=duration, n_steps=n_steps, LSTMBool=LSTMBool, predictDuration = predictDuration)
        predict3(noizeNet, predicted[-1] ,duration=duration, n_steps=n_steps, LSTMBool=LSTMBool, predictDuration = predictDuration,  hidden=hidden)

TRAINING...


/home/liam/anaconda3/envs/myenv/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/home/liam/anaconda3/envs/myenv/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Training Progress:	0.0%
P:	[-0.04941618  0.00057781 -0.07336345 -0.06047632 -0.05029624]
Y:	[0.78485084 0.8130057  0.83501863 0.84733    0.8533735 ]
X:	[0.7577045  0.78485084 0.8130057  0.83501863 0.84733   ]
Training Loss:  0.31421470642089844 Validation loss:  0.2561328113079071 	 num: 0 	 File: 1
Training Progress:	0.0%
P:	[0.5456647  0.6161146  0.6258562  0.6115618  0.60452753]
Y:	[0.63663906 0.655915   0.67280006 0.6856986  0.6932378 ]
X:	[0.6161611  0.63663906 0.655915   0.67280006 0.6856986 ]
Training Loss:  0.0031718339305371046 Validation loss:  0.0024695033207535744 	 num: 1000 	 File: 1
Training Progress:	0.0%
P:	[0.3591496  0.3174811  0.2810474  0.2648716  0.14796233]
Y:	[0.2353786  0.23967282 0.24724782 0.25629455 0.2662226 ]
X:	[0.23270741 0.2353786  0.23967282 0.24724782 0.25629455]
Training Loss:  0.0012190474662929773 Validation loss:  0.001719089224934578 	 num: 2000 	 File: 1
Training Progress:	0.0%
P:	[0.50193775 0.52162254 0.5291595  0.543864   0.56907594]
Y:	[0.52